In this file we will prepare our address information and commodity information datasets for evaluation.

In [75]:
import pandas as pd
import re

address_set = pd.read_csv('C://Users//stoandel//Documents//PythonProgramming//mdBest//address_info.csv', header=None)
commodity_set = pd.read_csv('C://Users//stoandel//Documents//PythonProgramming//mdBest//commodity_list.csv', header=None)

Unneccesary columns are dropped

In [76]:
address_set = address_set.drop([3,4,5], axis=1)

In [77]:
print(address_set.columns)

Int64Index([0, 1, 2], dtype='int64')


In [78]:
print(address_set.head)

<bound method NDFrame.head of        0                                                  1  \
0      1         5814 Broad Run Road, Jefferson, MD,  21755   
1      2            400 East 32nd St, Baltimore, MD,  00000   
2      3            3365 Hall Creek Lane, Owings, MD  20736   
3      4        4024 Main St, Grasonville, MD 21638, USA \n   
4      5  2001 Medical Parkway - South Tower, lower leve...   
..   ...                                                ...   
342  343      17051 Virginia Avenue, Williamsport, MD 21795   
343  344            8109 Telegraph Road, Severn, MD,  21144   
344  345            11939 Harford Road\n Glen Arm, MD 21057   
345  346       2115 Waverly Drive, Bel Air, MD,  21015-1013   
346  347     2108 Pleasant Valley Road, Oakland, MD,  21550   

                                         2  
0            39.389087, -77.59276999999997  
1                    39.327199, -76.610929  
2                    38.690406, -76.667956  
3                    38.958750, -

We will check if there are coordinates for each record. If some are missing, we may cross-reference using the address.

In [79]:
pattern = '[0-9]{2}.[0-9], [-0-9]'

# Not displayed
#print(address_set[address_set[2].str.contains(pattern) == False])

#Print how many are missing coordinates
print(len(address_set[address_set[2].str.contains(pattern) == False]))

# Copy the records missing coordinates to a new dataframe
coordinate_set = address_set[address_set[2].str.contains(pattern) == False]


21


Only 21 records do not have coordinates. Fortunately it appears that all except for two of the records not containing coordinates have an address (not displayed here).

To get the coordinates we will be utilizing [Google's Geocoding API](https://developers.google.com/maps/documentation/geocoding/overview). Originally the plan was to use the [U.S. Census Bureau's Geocoding Services API](https://geocoding.geo.census.gov/geocoder/Geocoding_Services_API.html); however, it did not do a good job of catching any name discrapencies. The basic examples provided in the API documentation have been adapted for this purpose.

In [26]:
import googlemaps
import time
county_list = list()
result_list = list()
gmaps = googlemaps.Client(key='')

delay = 0.1  # Set the delay to 100ms.

for i in coordinate_set[1]:
    # Geocoding an address
    geocode_result = gmaps.geocode(i)
    result_list.append(geocode_result)
    time.sleep(delay)

Maryland
Kent County
Carroll County
Montgomery County
Baltimore County
Caroline County
Prince George's County
Anne Arundel County
Cecil County
Carroll County
Anne Arundel County
Howard County
Maryland
Frederick County
Caroline County
Carroll County
Wicomico County
Montgomery County
Baltimore County
Garrett County
Frederick County


In [ ]:
# Extract the county name for each result
for i in result_list:
    for j in i[0]['address_components']:
        if j['long_name'] and 'County' in j['long_name']:
            county_list.append(j['long_name'])
            
# Extract the coordinates for each result (not needed, but good for completion)
coordinate_list = list()
for i in result_list:
    coordinate_list.append(str(i[0]['geometry']['location']['lat']) + ', ' + str(i[0]['geometry']['location']['lng']))

In [80]:
# Add the counties to a new column
address_set.loc[address_set[2].str.contains(pattern) == False, 3] = county_list

In [82]:
# Add in the newly found coordinates
address_set.loc[address_set[2].str.contains(pattern) == False, 2] = coordinate_list